In [23]:
# ! pip install ipynb

You should consider upgrading via the 'c:\users\esimonds\anaconda3\python.exe -m pip install --upgrade pip' command.


In [24]:
import pandas as pd
import json
import numpy as np
from time import time
from tkinter import Tk, messagebox
from tkinter.filedialog import askopenfilename
from string import ascii_uppercase as upstr
import ipynb.fs.full.file_split as fs
# import file_split as fs
from ipynb.fs.full.import_csv import FileFinder
import ipynb.fs.full.data_formatter as formatter
import ipynb.fs.full.enspire_od_join as enspire_od_join
import ipynb.fs.full.pt_calculations as pt_calculations
from ipynb.fs.full.import_od import import_od

TODO just create separate inputs module

In [25]:
def concat_projs(df):
    all_projs_df = pd.DataFrame()
    pass

In [26]:
def run_main(proj_dict):
    proj_concat = False
    file_finder = FileFinder()
    proj_dict = fs.split_projects(proj_dict)
    for proj, inner_dict in proj_dict.items():
        project_title = proj
        if "-" in proj:
            inner_dict["proj_name"] = proj.split("-")[0]
            inner_dict["ab_name"] = proj.split("-")[-1]
        else:
            inner_dict["ab_name"] = ""
            inner_dict["proj_name"] = proj
        proj_data = proj_dict[proj]
        source_df = file_finder.data_finder(proj_data)
        df_list = formatter.data_format(source_df, proj_data)
        if proj_data["od_file"]:
            raw_od = import_od(proj_data)
            joined_df_list = enspire_od_join.join_dfs(df_list, raw_od, proj_data)
            main_join_dfs = joined_df_list[:2]
            final_display_df = joined_df_list[2]
            final_display_rep_df = joined_df_list[3]
            completed_main_dfs = pt_calculations.make_calculations(main_join_dfs, proj_data)
        else:
            main_dfs = df_list[:2]
            final_display_df = df_list[2]
            final_display_rep_df = df_list[3]
            completed_main_dfs = pt_calculations.make_calculations(main_dfs, proj_data)
        final_main_df = completed_main_dfs[0]
        final_main_rep_df = completed_main_dfs[1]
        with pd.ExcelWriter(f"../../test_files/test_outputs/{project_title}_output_test.xlsx") as writer:
            final_main_df.to_excel(writer, sheet_name="Calculations")
            final_display_df.to_excel(writer, sheet_name="Display_Ready")
            final_main_rep_df.to_excel(writer, sheet_name="Rep_Calculations")
            final_display_rep_df.to_excel(writer, sheet_name="Rep_Display_Ready")
        print(f"Project {project_title} has been output.")

        # Also return these four dataframes into list?
        # clean_df, main_df, clean_rep_df, main_rep_df = test_formatter.data_format(source_df, proj_data)
        # df_list = [clean_df, main_df, clean_rep_df, main_rep_df]
        # dfs_return = join_dfs(df_list, raw_od, proj_data)
        #
        # clean_join_df = dfs_return[0]
        # main_join_df = dfs_return[1]
        # clean_rep_join_df = dfs_return[2]
        # main_rep_join_df = dfs_return[3]
        #
        # complete_df = eight_pt_calculations.make_calculations(main_join_df, proj_data)
        # complete_rep_df = eight_pt_calculations.make_calculations(main_rep_join_df, proj_data)
        #

In [27]:
if __name__ == "__main__":
    start_time = time()
    window = Tk()
    window.withdraw()
    try:
        with open(r"C:\Users\esimonds\parser_data.json", "r") as parser_file:
            proj_data_dict = json.load(parser_file)
    except FileNotFoundError:
        messagebox.showinfo(title="Warning!", message="There's no file, did you complete the parser form?")

    run_main(proj_data_dict)
    window.destroy()
    end_time = time()
    split = round(end_time - start_time, 2)
    print(f"Program runtime: {split}s")

SOM00012 and 2
Project SOM00012 has been output.
Program runtime: 19.09s
